In [1]:
from edc import print_info
print_info("geodb-sharing")


***Notebook Title***  
GeoDB: Sharing Data

***Notebook Description***  
Euro Data Cube GeoDB: Sharing Data


***Notebook Dependencies***  
This notebook requires an active subscription to:
* EDC GeoDB


In [2]:
from edc import setup_environment_variables
setup_environment_variables()

API credentials have automatically been injected for your active subscriptions.  
The following environment variables are now available:
* `GEODB_API_SERVER_PORT`, `GEODB_API_SERVER_URL`, `GEODB_AUTH_AUD`, `GEODB_AUTH_CLIENT_ID`, `GEODB_AUTH_CLIENT_SECRET`, `GEODB_AUTH_DOMAIN`

The following additional environment variables have been loaded from `~/custom.env`:
* `AWS_BUCKET`
* `DAPA_URL`
* `DB_HOST`, `DB_NAME`, `DB_PASSWORD`, `DB_USER`
* `OGC_EDC_URL`
* `REFERENCE_DATA`


In [3]:
from edc import check_compatibility
check_compatibility("user-0.23.1")

## Sharing Data

In [4]:
from xcube_geodb.core.geodb import GeoDBClient

In [5]:
geodb = GeoDBClient()
geodb.whoami

'geodb_418dfeac-15f0-4606-9edb-fd9eb722bf04'

In [6]:
ds = geodb.get_my_collections()

if ds[(ds.database == geodb.whoami) & (ds.table_name == 'land_use')].table_name.count() == 0:
    # Have a look at fiona feature schema
    collections = {
            "land_use": 
            {
                "crs": 3794,
                "properties": 
                {
                    "RABA_PID": "float", 
                    "RABA_ID": "float", 
                    "D_OD": "date"
                }
            }
        }


    geodb.create_collections(collections)
    import geopandas
    import os
    gdf = geopandas.read_file(os.path.expanduser("~/.shared/notebooks/eurodatacube/notebooks/getting-started/data/sample/land_use.shp"))
    geodb.insert_into_collection('land_use', gdf)

### Publish a Collection to the World

In [7]:
geodb.list_my_grants()

,database,table_name,grantee,privileges
0,geodb_418dfeac-15f0-4606-9edb-fd9eb722bf04,land_use,PUBLIC,SELECT


In [8]:
geodb.publish_collection("land_use")

Access granted on land_use to public.

### Accessing Collection as a different User

Please be aware that you cannot change the user easily within a Notebook. Here, we changed the credentials in the background. The following cells will not run and are for demonstration only.

In [9]:
geodb = GeoDBClient()
geodb.whoami

'geodb_418dfeac-15f0-4606-9edb-fd9eb722bf04'

In [10]:
# geodb.get_collection('land_use', database='geodb_admin')

### Revoke access

In [11]:
geodb.list_my_grants()

,database,table_name,grantee,privileges
0,geodb_418dfeac-15f0-4606-9edb-fd9eb722bf04,land_use,PUBLIC,SELECT


In [12]:
geodb.unpublish_collection("land_use")

Access revoked from user public on land_use

In [13]:
geodb.list_my_grants()

,Grants
0,No Grants


In [14]:
gdf = geodb.get_collection_by_bbox(collection="land_use", bbox=(452750.0, 88909.549, 464000.0, 102486.299),
                comparison_mode="contains", bbox_crs=3794, limit=2, offset=10)
gdf

,id,created_at,modified_at,geometry,raba_pid,raba_id,d_od
0,12,2020-09-29T09:23:21.172369+00:00,None,"POLYGON ((453673.609 91328.224, 453678.929 913...",5983074,1600,2019-03-26
1,13,2020-09-29T09:23:21.172369+00:00,None,"POLYGON ((460312.295 96127.114, 460300.319 961...",5983199,1600,2019-03-11
